In [10]:
import pandas as pd

data = pd.read_csv('~/ctgan/data/annualized_SD_p8_std_except_Highmore.csv')
data.head()

,Annual Avg Yield,Annual Avg Min Temp,Annual Avg Max Temp,Total Accumulated Rain,Total Accumulated Radiation,Class,avg,std,num stds
0,0.526667,2.882998,14.956573,2064.15,419077.34,0,1.452725,0.429985,-2.153698
1,0.523333,2.882998,14.956573,2064.15,419077.34,0,1.452725,0.429985,-2.161450
2,0.456667,2.882998,14.956573,2064.15,419077.34,0,1.452725,0.429985,-2.316494
3,0.530000,2.882998,14.956573,2064.15,419077.34,0,1.452725,0.429985,-2.145946
4,0.600000,2.882998,14.956573,2064.15,419077.34,0,1.452725,0.429985,-1.983150


In [11]:
from sdv.tabular import CTGAN

model = CTGAN()
model.fit(data)

In [12]:
new_data = model.sample(200000)

In [13]:
new_data.to_csv('~/ctgan/data/synthData/xgb_1101_200k_snlt_sd_ann2hmsd8_tvae1.csv')
new_data.head()

,Annual Avg Yield,Annual Avg Min Temp,Annual Avg Max Temp,Total Accumulated Rain,Total Accumulated Radiation,Class,avg,std,num stds
0,1.877665,1.714360,14.981881,3544.42,646196.0,1,1.452725,0.429985,0.718937
1,1.256548,3.182788,15.244523,6492.45,799163.0,1,1.452725,0.429985,1.240742
2,2.380691,1.869228,15.473735,2461.09,834109.0,0,1.452725,0.429985,-0.057188
3,1.829873,2.485244,15.871712,3144.02,1123306.0,1,1.452725,0.429985,0.089435
4,1.802222,1.639312,15.451880,2117.64,1501924.0,1,1.452725,0.429985,0.038521


In [14]:
# get aggregate data
targetDataLoc = '~/ctgan/data/annualized_SD_p8_std_Highmore_8.csv'
#aggDataLoc = 'data/synth1_GA_only_063022.csv'

aggDf = new_data #pd.read_csv(aggDataLoc)
#aggDf = aggDf.drop("Unnamed: 0",axis=1)
targetDf = pd.read_csv(targetDataLoc)
#targetDf = targetDf.drop("Unnamed: 0",axis=1)

In [15]:
############## imports
# general
import statistics
import datetime
#from sklearn.externals import joblib # save and load models
import random
# data manipulation and exploration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

## machine learning stuff
# preprocessing
from sklearn import preprocessing
# feature selection
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_regression
# pipeline
from sklearn.pipeline import Pipeline
# train/testing
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score  
# error calculations
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# models
from sklearn.linear_model import LinearRegression # linear regression
from sklearn.linear_model import LogisticRegression # logistic regression
from sklearn.linear_model import BayesianRidge #bayesisan ridge regression
from sklearn.svm import SVR  # support vector machines regression
from sklearn.svm import SVC  # support vector machines classification
from sklearn.gaussian_process import GaussianProcessRegressor # import GaussianProcessRegressor
from sklearn.neighbors import KNeighborsRegressor # k-nearest neightbors for regression
from sklearn.neighbors import KNeighborsClassifier # k-nearest neightbors for classification
from sklearn.neural_network import MLPRegressor # neural network for regression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeRegressor # decision tree regressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor  # random forest regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostRegressor # adaboost for regression
import xgboost as xgb
# saving models
# from sklearn.externals import joblib
import joblib

# import the API
APILoc = 'API/'

import sys
sys.path.insert(0, APILoc)

from API import *

In [16]:
# filter out the features that will not be used by the machine learning models

# the features to keep:
# xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Time Since Last Harvest (Days)", "Total Radiation (MJ/m^2)",
#                "Total Rainfall (mm)", "Avg Air Temp (C)", "Avg Min Temp (C)", "Avg Max Temp (C)",
#                  "Avg Soil Moisture (%)", "Day Length (hrs)"], "Percent Cover (%)"]

# xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Total Radiation (MJ/m^2)",
#                "Total Rainfall (mm)", "Avg Air Temp (C)", "Avg Min Temp (C)", "Avg Max Temp (C)",
#                  "Avg Soil Moisture (%)"]

xColumnsToKeep = ["Total Accumulated Radiation","Total Accumulated Rain", "Annual Avg Max Temp", "Annual Avg Min Temp"]


#xColumnsToKeep = ["Julian Day", "Time Since Sown (Days)", "Total Radiation (MJ/m^2)", "Total Rainfall (mm)"]

# the target to keep
yColumnsToKeep = ["Class"]

# get a dataframe containing the features and the targets
xDf = aggDf[xColumnsToKeep]
test_xDf = targetDf[xColumnsToKeep]
yDf = aggDf[yColumnsToKeep]
test_yDf = targetDf[yColumnsToKeep]

# reset the index
xDf = xDf.reset_index(drop=True)
yDf = yDf.reset_index(drop=True)
test_xDf = test_xDf.reset_index(drop=True)
test_yDf = test_yDf.reset_index(drop=True)

pd.set_option('display.max_rows', 2500)
pd.set_option('display.max_columns', 500)

xCols = list(xDf)

In [17]:
# hide the warnings because training the neural network caues lots of warnings.
import warnings
warnings.filterwarnings('ignore')

# make the parameter grids for sklearn's gridsearchcv
rfParamGrid = {
        'model__n_estimators': [5, 10, 25, 50, 100], # Number of estimators
        'model__max_depth': [5, 10, 15, 20], # Maximum depth of the tree
        'model__criterion': ["gini"]
    }
knnParamGrid ={
        'model__n_neighbors':[2,5,10],
        'model__weights': ['uniform', 'distance'],
        'model__leaf_size': [5, 10, 30, 50]    
    }
svrParamGrid = {
        'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'model__C': [0.1, 1.0, 5.0, 10.0],
        'model__gamma': ["scale", "auto"],
        'model__degree': [2,3,4,5]
    }
nnParamGrid = {
        'model__hidden_layer_sizes':[(3), (5), (10), (3,3), (5,5), (7,7)],
        'model__solver': ['sgd', 'adam'],
        'model__learning_rate' : ['constant', 'invscaling', 'adaptive'],
        'model__learning_rate_init': [0.1, 0.01, 0.001]      
    }

linRegParamGrid = {}

bayesParamGrid={
        'model__n_iter':[100,300,500]
    }

dtParamGrid = {
    'model__criterion': ['gini'],
    'model__max_depth': [5,10,25,50,100]
    }

xgbParamGrid = {
    #'tree_method': ["gpu_hist"], 
    #'enable_categorical': [True], 
    #'use_label_encoder': [False]
}

aModelList = [#(RandomForestClassifier(), rfParamGrid, "rfTup.pkl"),
              #(KNeighborsClassifier(), knnParamGrid, "knnTup.pkl"),
              #(SVC(), svrParamGrid, "svrTup.pkl"),
             #(MLPClassifier(), nnParamGrid, "nnTup.pkl")]#,
             #(LogisticRegression(), linRegParamGrid, "linRegTup.pkl")]#,
             #(BayesianRidge(), bayesParamGrid, "bayesTup.pkl"),
             #(DecisionTreeClassifier(), dtParamGrid, "dtTup.pkl")]
             (xgb.XGBClassifier(), xgbParamGrid, "xgbTup.pkl")]

N = 10
workingDir = 'working_dir'
numFeatures = 4 # 11

In [18]:
saveMLResults(test_xDf, test_yDf, N, xDf, yDf, aModelList, workingDir, numFeatures, printResults=True)

model:  xgbTup
Avg MAE:  427.66
Avg R squared:  -0.013
Best MAE:  434.3
Best R squared:  -0.011
Parameters of the best model:  {}
Features selected by best model:  ['Total Accumulated Radiation', 'Total Accumulated Rain', 'Annual Avg Max Temp', 'Annual Avg Min Temp']
Accuracy:  0.78705
f1 score:  0.29361237794129996
mcc:  -0.002902379282267667
 
test results on our test data: 
results:
MAE:  0.5
R sq:  0.0
accuracy score for DA/TDA:  0.5
